In [2]:
import nltk
import random

In [3]:
BOT_CONFIG = {
    "intents": { # Намерения
        "hello": { # Намерение поздароваться
            "examples": ["Привет", "Добрый день", "Шалом", "Здравствуйте"],
            "responses": ["Привет, человек", "Доброго времени суток"]
        },
        "bye": {
            "examples": ["Пока", "Досвидос", "Прощай"],
            "responses": ["Счастливо", "До свидания", "Если что, возвращайтесь"],
        },
        "howdoyoudo": {
            "examples": ["Как дела", "Что делаешь", "Какие дела"],
            "responses": ["Маюсь фигней", "Отвечаю на дурацкие вопросы", "Веду вебинары"],
        },
    },
    "failure_phrases": [
        "Я ничо не понил",
        "Что-то непонятно",
        "Я всего лишь бот, сформулируйте попроще"
    ]
}

In [5]:
def filter(text):
  text = text.lower()
  text = [c for c in text if c in 'абвгджзеёийклмнопрстуфхцчшщьыъэюя -']
  return ''.join(text)

def match(text, example): # "прощяй!" === "Прощай" ??
  text = filter(text)
  #example = filter(example)
  
  distance = nltk.edit_distance(text, example) / len(example)
  if distance < 0.4:
    return True # Текст совпадает
  else:
    return False # Текст НЕ совпадает

In [6]:
def get_intent(text): 
  for intent, data in BOT_CONFIG['intents'].items():
    for example in data['examples']:
      if match(text, example):
        return intent

In [7]:
def get_answer_by_intent(intent):
  phrases = BOT_CONFIG['intents'][intent]['responses']
  return random.choice(phrases)

In [9]:
def bot(text):
  # 1. Понять намерение
  intent = get_intent(text)

  if not intent:
    intent = get_intent_predictive_model(text)

  if intent:
    return get_answer_by_intent(intent)

  

  # 3. Отвечаем "заглушкой"
  failure_phrases = BOT_CONFIG['failure_phrases']
  return random.choice(failure_phrases)

In [10]:
question = ''
while question not in ['выход', 'отстань']:
  question = input()
  answer = bot(question)
  print(answer)

выход


NameError: ignored

In [11]:
import json

In [12]:
config_file = open('/content/big_bot_config.json', "r")

In [13]:
BOT_CONFIG = json.load(config_file)

In [14]:
len(BOT_CONFIG['intents'])

439

In [15]:
BOT_CONFIG['intents']['hello']['examples']

['Привет честной компании',
 'Нихао',
 'Привет машина',
 'Хеллоу',
 'Вопрос. Сколько стоит обучение?',
 'Здравия желаю',
 'Приветствую Вас',
 'алоха',
 'Привяу',
 'Дратути',
 'Приветули',
 'Приветствую!',
 'рад видеть снова',
 'приятно видеть тебя снова',
 'Доброе утро',
 'привет',
 'Сколько лет сколько зим',
 'Хэллоу',
 'Здравствуйте',
 'Hi',
 'Хэлоу',
 'Приветухи',
 'Добрый день!',
 'Здорово, чурбан электронный',
 'hi',
 'Рад приветствовать Вас',
 'салют',
 'Добрый вечер',
 'Чё кого',
 'Даров',
 'Hello',
 'Hola',
 'Дозвольте приветствовать Вас',
 'Привет',
 'Вечер в хату',
 'Приветствую вас',
 'Какие люди',
 'гутентаг',
 'здарямба',
 'Доброго времени суток!',
 'здорова',
 'Эрик?',
 'Доброго здравия',
 'хай',
 'Приветики',
 'салям',
 'Бонжур',
 'Бодрого утречка',
 'Салют!Здоровеньки булы',
 'А что мы тут делаем',
 'Доброго утра',
 'Доброй ночи',
 'здрасьте',
 'простоздравствуй',
 'Дороу',
 'Салям Алейкум',
 'моё почтение',
 'здравствуйте',
 'Здорово',
 'хело',
 'Доброго здоровья',
 'С

In [16]:
X_examples = []
y = []
for intent, data in BOT_CONFIG['intents'].items():
  for example in data['examples']:
    X_examples.append(example)
    y.append(intent)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
count_vectorizer.fit(X_examples)

In [18]:
# list(count_vectorizer.transform(['чем ты там занимаешься']).nonzero()[1])

In [19]:
X = count_vectorizer.transform(X_examples)

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
log_reg = LogisticRegression()
log_reg.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
log_reg.predict(count_vectorizer.transform(['чем ты там занимаешься']))

array(['byflood'], dtype='<U29')

In [23]:
log_reg.score(X, y)

0.38840772818121255

In [24]:
from sklearn.svm import LinearSVC

In [25]:
# X_train Тренировочная выборка
# X_test тестовая выборка

In [26]:
from sklearn.feature_extraction.text import HashingVectorizer

In [27]:
hashing_vectorizer = HashingVectorizer()
X = hashing_vectorizer.fit_transform(X_examples)


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
TFV = TfidfVectorizer(analyzer='char_wb', ngram_range=(2,4))
X = TFV.fit_transform(X_examples)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33
)

In [32]:
lin_svc = LinearSVC()
lin_svc.fit(X_train, y_train)
print ('Train', lin_svc.score(X_train, y_train))
print ('Test', lin_svc.score(X_test, y_test))

Train 0.8587767279960219
Test 0.3410696266397578


In [33]:
def get_intent_predictive_model(text):
  return lin_svc.predict(TFV.transform([text]))[0]

In [34]:
import pickle

In [35]:
pickle.dump(lin_svc, open('lin_svc.model', 'wb'))
pickle.dump(TFV, open('TFV.model', 'wb'))

In [36]:
vctrzr = pickle.load(open('lin_svc.model', 'rb'))

In [37]:
! pip install python-telegram-bot --upgrade

     |████████████████████████████████| 450kB 16.5MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 


In [38]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [39]:
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    update.message.reply_text('Кукуськи, пуська!')


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    answer = bot(update.message.text)
    update.message.reply_text(answer)


def main():
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("1311062782:AAHVB0rcTMuXQjIBLSO9IJ1IzD3hFSfszhg", use_context=True)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on noncommand i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
main()